## 1. Setup Development Environment



In [ ]:
!pip install googletrans==3.1.0a0 datasets

## Connect to Drive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load Dataset

In [ ]:
def create_train_test(dataset):
    questions, answers = [], []
    for record in dataset['data']:
        title = record['title']
        paragraph = record['paragraphs'][0]
        context = paragraph['context']
        for q in paragraph['qas']:
            question = q['question']
            try: short_answer = q['answers'][0]['text']
            except: short_answer = "نامشخص"
            questions.append(question)
            answers.append(f"Title: {title}\nShort Answer: {short_answer}\nLong Answer: {context}")
    df = pd.DataFrame({'question': questions, 'answer': answers})
    return df

In [ ]:
import json

f = open('/content/drive/MyDrive/Corpus/PersianQA/pqa_train.json')
train_set = json.load(f)
f = open('/content/drive/MyDrive/Corpus/PersianQA/pqa_test.json')
test_set = json.load(f)

df_train = create_train_test(train_set)
df_test = create_train_test(test_set)

In [ ]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 9008
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 930
    })
})

In [ ]:
# num_train_samples = 1000
# num_test_samples = 100

# train_dataset = dataset['train'].shuffle(seed=42).select(range(num_train_samples))
# test_dataset = dataset['test'].shuffle(seed=42).select(range(num_test_samples))

# dataset = DatasetDict({
#     "train": train_dataset,
#     "test": test_dataset
# })
# dataset

In [ ]:
import googletrans
from googletrans import Translator

translator = Translator()

def translated_text(input_text):
    translated = translator.translate(input_text, src='fa', dest='en')
    return translated.text

In [ ]:
from tqdm import tqdm

def translated_df(dataset):
    questions, answers, questions_en, answers_en = [], [], [], []
    for i in tqdm(range(len(dataset['question'])), desc="Translating..."):
        questions.append(dataset['question'][i])
        answers.append(dataset['answer'][i])
        questions_en.append(translated_text(dataset['question'][i]))
        answers_en.append(translated_text(dataset['answer'][i]))
    df = pd.DataFrame({'question': questions, 'answer': answers, 'question_en': questions_en, 'answer_en': answers_en})
    return df

In [ ]:
df_train = translated_df(dataset['train'])
df_test = translated_df(dataset['test'])
dataset = DatasetDict({"train": Dataset.from_pandas(df_train), "test": Dataset.from_pandas(df_test)})

In [ ]:
df_train

,question,answer,question_en,answer_en
0,شرکت فولاد مبارکه در کجا واقع شده است,Title: شرکت فولاد مبارکه اصفهان\nShort Answer:...,Where is Mobarake Steel Company located?,Title: Isfahan Mobarakeh Steel Company\nShort ...
1,فولاد مبارکه چند بار برنده جایزه شرکت دانشی را...,Title: شرکت فولاد مبارکه اصفهان\nShort Answer:...,How many times has Mobarakeh Steel won the pri...,Title: Isfahan Mobarakeh Steel Company\nShort ...
2,شرکت فولاد مبارکه در سال ۱۳۹۱ چه جایزه ای برد؟,Title: شرکت فولاد مبارکه اصفهان\nShort Answer:...,What award did Mobarake Steel Company win in 2...,Title: Isfahan Mobarakeh Steel Company\nShort ...
3,بزرگ ترین مجموعه تولید فولاد ایران چیست؟,Title: شرکت فولاد مبارکه اصفهان\nShort Answer:...,What is the largest steel production complex i...,Title: Isfahan Mobarakeh Steel Company\nShort ...
4,فولاد مبارکه در چه سالی احداث شد؟,Title: شرکت فولاد مبارکه اصفهان\nShort Answer:...,In what year was Mobarake Steel built?,Title: Isfahan Mobarakeh Steel Company\nShort ...
...,...,...,...,...
9003,همهٔ نخستیان به جز کدام موجود دارد شارب هستند؟,Title: شارب\nShort Answer: انسان\nLong Answer:...,All primates are drinkers except which one?,Title: Drink\nShort Answer: Man\nLong Answer: ...
9004,شارب را می توان در کدام موجودات پیدا کرد؟,Title: شارب\nShort Answer: پستاندارانی چون سگ،...,Sharb can be found in which creatures?,Title: Drink\nShort Answer: Mammals such as do...
9005,عملکرد شارب در جانوران بی مهره چه گونه است؟,Title: شارب\nShort Answer: نامشخص\nLong Answer...,What is the function of drinking in invertebra...,Title: Drink\nShort Answer: Uncertain\nLong An...
9006,چرا شارب در بعضی هز جانوران خیلی مهم نیست؟,Title: شارب\nShort Answer: نامشخص\nLong Answer...,Why is wine not very important in some animals?,Title: Drink\nShort Answer: Uncertain\nLong An...


In [ ]:
import pickle

f = open('persian_question_answering.dat', 'wb')
pickle.dump(dataset, f)

In [ ]:
dataset['train'][65]

{'question': 'دلیل وجود نشان شیطان در لوگو باشگاه منچستر چیه؟',
 'answer': 'Title: باشگاه فوتبال منچستر یونایتد\nShort Answer: در دههٔ ۶۰ میلادی، به منچستر یونایتد لقب شیاطین سرخ داده شد، در آن زمان این لقب به صورت رسمی وارد برنامه\u200cهای باشگاه شد و در دههٔ ۷۰ میلادی، نشان یک شیطان به آرم باشگاه اضافه شد\nLong Answer: باشگاه فوتبال منچستر یونایتد یکی از باشگاه\u200cهای حرفه\u200cای فوتبال در لیگ برتر فوتبال انگلستان است. این باشگاه در سال ۱۸۷۸، با نام باشگاه فوتبال نیوتون هیث تأسیس شد. در سال ۱۹۰۲، نام این باشگاه به منچستر یونایتد تغییر یافت و در سال ۱۹۱۰، ورزشگاه الدترافورد به عنوان ورزشگاه خانگی این تیم انتخاب شد. منچستر یونایتد با داشتن بیشترین هوادار، باشگاه\u200cهای هواداری و بیشترین میانگین تماشاگر برای هر بازی خانگی، پرطرفدارترین تیم جهان به حساب می\u200cآید. این باشگاه بالغ بر ۲۰۰ باشگاه هواداری رسمی را در ۲۴ کشور دنیا اداره می\u200cکند. باشگاه منچستر یونایتد به خاطر تورهای تابستانی خود و سفر به نقاط مختلف جهان در تعطیلات، محبوبیت ویژه\u200cای در نزد مردم جهان دارد. نتایج یک

In [ ]:
from random import randrange


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"question: \n{sample['question']}\n---------------")
print(f"title: \n{sample['title']}\n---------------")
print(f"answer: \n{sample['answer']}\n---------------")
print(f"context: \n{sample['context']}\n---------------")